In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from google.colab import drive
import nltk
from textblob import TextBlob
from datetime import datetime
drive.mount('/content/drive/', force_remount=True)
nltk.download('stopwords')
stop_words = stopwords.words('english')

sentiment_map = list()

Mounted at /content/drive/
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
'''For Year 2019'''
entity = list()
for index in range(1, 12):
  item = dict()
  if index == 4 or index == 6 or index == 9 or index == 11:
    entity.append({'filename' :'df_tweet_file_data_' + str(index) + '.csv','month': index, 'days': 30})
  elif index == 2:
    entity.append({'filename' :'df_tweet_file_data_' + str(index) + '.csv','month': index, 'days': 28})
  else:
    entity.append({'filename' :'df_tweet_file_data_' + str(index) + '.csv','month': index, 'days': 31}) 

In [ ]:
def preprocess(df_crypto):
  df_crypto['datetime'] =  pd.to_datetime(df_crypto['timestamp'], format='%Y-%m-%d %H:%M:%S+%f')
  df_crypto['text'] = df_crypto['text'].astype(str).apply(lambda x: re.sub('[^a-zA-Z0-9]+', ' ', x).lower())
  df_crypto['text'] = df_crypto['text'].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
  df_crypto['sentiment'] = df_crypto['text'].apply(lambda x: TextBlob(x).sentiment[0])
  return df_crypto[['datetime', 'sentiment']]

In [ ]:
def get_avg(hour_df) -> 'sentiment_average':
  negative_count = [item for item in hour_df['sentiment'] if item < 0.0]
  positive_count = [item for item in hour_df['sentiment'] if item > 0.0]
  if len(negative_count) > len(positive_count):
    average = sum(negative_count)/len(negative_count)
    return average
  else:
    if len(positive_count) > 0:
      average = sum(positive_count)/len(positive_count)
      return average
    else:
      return 0

In [ ]:
year = 2019
hour_range = [item for item in range(0, 24)]
for item in entity:
  print('Processing for '+item['filename'])
  df_raw = pd.read_csv('/content/drive/My Drive/Tweet_Data/df_tweet_data_2019/'+ item['filename'])
  date_range = [item for item in range(1, item['days'] + 1)]
  df_senti = preprocess(df_raw)
  for date_stamp in date_range:
    for hour_stamp in hour_range:
      starttime = str(year) + '-' + str(item['month']) + '-' + str(date_stamp)+ ' ' +  str(hour_stamp) + ':00:00'
      if hour_stamp == 23:
        endtime = str(year) + '-' + str(item['month']) + '-' + str(date_stamp)+ ' ' + str(hour_stamp) + ':59:59'
      else:
        endtime = str(year) + '-' + str(item['month']) + '-' + str(date_stamp)+ ' ' +  str(hour_stamp + 1) + ':00:00'
      start = datetime.strptime(starttime, '%Y-%m-%d %H:%M:%S')
      end = datetime.strptime(endtime, '%Y-%m-%d %H:%M:%S')
      mask = (df_senti['datetime'] >= start) & (df_senti['datetime'] < end)
      date_range_df = df_senti.loc[mask]
      avg_val = get_avg(date_range_df)
      sentiment_map.append({'timestamp': end, 'sentiment':avg_val})

final_df = pd.DataFrame(sentiment_map)
final_df.to_csv('/content/drive/My Drive/CA683/tweet_2019_sentiment.csv', sep=',')

Processing for df_tweet_file_data_1.csv
Processing for df_tweet_file_data_2.csv
Processing for df_tweet_file_data_3.csv
Processing for df_tweet_file_data_4.csv
Processing for df_tweet_file_data_5.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Processing for df_tweet_file_data_6.csv
Processing for df_tweet_file_data_7.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Processing for df_tweet_file_data_8.csv
Processing for df_tweet_file_data_9.csv
Processing for df_tweet_file_data_10.csv
Processing for df_tweet_file_data_11.csv


In [ ]:
'''For years 2009 to 2018'''
def monthentity():
  entity = list()
  for index in range(1, 13):
    item = dict()
    if index == 4 or index == 6 or index == 9 or index == 11:
      entity.append({'month': index, 'days': 30})
    elif index == 2:
      entity.append({'month': index, 'days': 28})
    else:
      entity.append({'month': index, 'days': 31}) 
  return entity

month_day = monthentity()

In [ ]:
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018] 

hour_range = [item for item in range(0, 24)]

for year in years:
  print('Processing for year '+ str(year))
  df_raw = pd.read_csv('/content/drive/My Drive/Tweet_Data/df_tweet_file_data_'+ str(year) + '.csv')
  df_senti = preprocess(df_raw)
  for item in month_day:
    date_range = [item for item in range(1, item['days'] + 1)]
    for date_stamp in date_range:
      for hour_stamp in hour_range:
        starttime = str(year) + '-' + str(item['month']) + '-' + str(date_stamp)+ ' ' +  str(hour_stamp) + ':00:00'
        if hour_stamp == 23:
          endtime = str(year) + '-' + str(item['month']) + '-' + str(date_stamp)+ ' ' + str(hour_stamp) + ':59:59'
        else:
          endtime = str(year) + '-' + str(item['month']) + '-' + str(date_stamp)+ ' ' +  str(hour_stamp + 1) + ':00:00'
        start = datetime.strptime(starttime, '%Y-%m-%d %H:%M:%S')
        end = datetime.strptime(endtime, '%Y-%m-%d %H:%M:%S')
        mask = (df_senti['datetime'] >= start) & (df_senti['datetime'] < end)
        date_range_df = df_senti.loc[mask]
        avg_val = get_avg(date_range_df)
        sentiment_map.append({'timestamp': end, 'sentiment':avg_val})

final_df = pd.DataFrame(sentiment_map)
final_df.to_csv('/content/drive/My Drive/CA683/tweet_2009_2018_sentiment.csv', sep=',')

Processing for year 2009
Processing for year 2010
Processing for year 2011
Processing for year 2012
Processing for year 2013
Processing for year 2014
Processing for year 2015
Processing for year 2016
Processing for year 2017


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Processing for year 2018


In [ ]:
df_09_18 = pd.read_csv('/content/drive/My Drive/CA683/tweet_2009_2018_sentiment.csv')
df_19 = pd.read_csv('/content/drive/My Drive/CA683/tweet_2019_sentiment.csv')

In [ ]:
frames = [df_09_18, df_19]
result = pd.concat(
    frames,
    axis=0,
    join="outer",
    ignore_index=True,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)
result.to_csv('/content/drive/My Drive/CA683/tweet_final_sentiment.csv', sep=',')